<a href="https://colab.research.google.com/github/nicolejlaurin/A-Star-Search-/blob/main/COMP4106_A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#COMP4106 Assignment 1

class node:
    def __init__(self, coords, value): #constructor to initilize values for node objects
        self.coords = coords
        self.value = value
        self.pathcost = 0
        self.totalcost = 0
        self.adjacent = []
        self.parent = None
        self.isExplored = False
    def __repr__(self):
        return '(' + ','.join(map(str,self.coords)) + ') : ' + str(list(map(str, self.adjacent)))
    def __str__(self):
        return '(' + ','.join(map(str,self.coords)) + ')' 
    def addAdjacent(self, node):      #adds nodes to adjacency list if it is not a wall
        if node.value != 'X' and node.value != 'x':
            self.adjacent.append(node)

def pathfinding(CSV, optimal, explored): # calls other functions and writes to the files
  #initialize empty lists to populate
  nodeArray = []
  goalArray = []
  exploredList = []
  frontier = []

  startNode = format_grid(CSV, nodeArray, goalArray)  #to format grid from input and get start node
  goalNode = traverse_grid(startNode, goalArray, frontier, exploredList) #traverse through grid to get goal node


  with open(explored, "w") as exploredFile:   #writes explored nodes to file and formats it
    for exp in exploredList:
      exploredFile.write(str(exp) + '\n')
    exploredFile.close()
  
  optimalList = [goalNode]
  currNode = goalNode
  optimalCost = 0

  while currNode.parent != None:    #determines optimal path from goal to start
    if currNode.value != 'g' and currNode.value != 'G':
      optimalCost += int(currNode.value)
    optimalList.append(currNode.parent)
    currNode = currNode.parent
  optimalList.reverse()             #reverse to format nodes start to goal

  with open(optimal, "w") as optimalFile:   #writes optimal path nodes to file and formats it
    for opt in optimalList:
      optimalFile.write(str(opt) + '\n')
    optimalFile.close() 
  return optimalCost          #returns cost of the optimal path



def traverse_grid(node, goalList, frontier, explored): #function to traverse to other nodes.


  explored.append(node)
  node.isExplored = True

  if node.value == 'g' or node.value == 'G':   #checks if current node is the Goal node
    return node
  else:     # if its not a goal node, it iterates through grid
    for adj in node.adjacent:
      if adj.pathcost > -1 and not adj.isExplored and pathcost(adj, node) < adj.pathcost: #if node is in frontier and total path cost is less than path cost for adjacent node
        frontier.remove(adj)
        #Need to also update entire subtree's pathcosts
      if adj.pathcost > -1 and adj.isExplored and pathcost(adj, node) < adj.pathcost: #if node is in explored and total path cost is less than path cost for adjacent node
        adj.isExplored = False
        explored.remove(adj)

      if not adj.isExplored and not adj in frontier: #checks if node is neither in explored or frontier
        adj.pathcost = pathcost(adj, node)
        adj.parent = node
        adj.totalcost = node_cost(adj, node, goalList)
        frontier.append(adj)

    frontier = sorted(frontier, key= lambda node : node.totalcost) 
    return traverse_grid(frontier.pop(0), goalList, frontier, explored)

def node_cost(node, parent, goalList): #outputs total cost to traverse to node
  
  step_cost = 0
  if node.value != 'g' and node.value != 'G' \    #checks node isnt the start or goal node
    and node.value != 's' and node.value != 'S':
    step_cost = int(node.value)

  path_cost = pathcost(node,parent)           #gets path cost from node to its parent
  heuristic_cost = manhattan(node, goalList)  #calculates the heuristic cost
  return path_cost + heuristic_cost 

def pathcost(node, parent): #outputs the path cost from parent node to next node
  step_cost = 0
  if node.value != 'g' and node.value != 'G' \     
    and node.value != 's' and node.value != 'S':
    step_cost = int(node.value)

  return parent.pathcost + step_cost

def manhattan(node, goalList):   #heuristic function to calculate distance between 2 nodes
  goalHeuristics = []
  for goal in goalList:
    goalHeuristics.append(abs(node.coords[0] - goal.coords[0]) + abs(node.coords[1] - goal.coords[1]))
  return min(goalHeuristics)    #returns minimum manhattan distance to any goal node

def format_grid(filePath, nodeArray, goalArray): #gets input text and formats it


  #Calculates the adjacent nodes for each spot in the grid and store in array
  #Output: array of arrays of nodes

  with open(filePath, encoding="utf8") as fp: #Reads the input CSV and creates lists of nodes for traversal 
    row = 0
    line = fp.readline()
   
    while line:
      nodeArray.append([])
      processed = line.strip().split(',')   #creates a node for every character in CSV
      for col in range(0, len(processed)):      #format grid and initialize nodes
        currnode = node((row, col), processed[col].strip())
        currnode.pathcost = -1
        nodeArray[row].append(currnode)
        if currnode.value == 's' or currnode.value == 'S':
          currnode.pathcost = 0
          startNode = currnode                    #initializing start node
        if currnode.value == 'g' or currnode.value == 'G':
          goalArray.append(currnode)             #adding goal node to list of all goals

        if col > 0:                         #Setting up adjacency lists of nodes to traverse grid 
          leftnode = nodeArray[row][col-1]
          leftnode.addAdjacent(currnode)
          currnode.addAdjacent(leftnode)
        if row > 0:
          uppernode = nodeArray[row-1][col]
          uppernode.addAdjacent(currnode)
          currnode.addAdjacent(uppernode)

      row += 1
      line = fp.readline()

  return startNode


pathfinding("/input.txt", "/optimal_path.txt", "/explored_list.txt")

Explored:  [(0,0) : ['(0,1)', '(1,0)'], (0,1) : ['(0,0)', '(0,2)'], (1,0) : ['(0,0)', '(2,0)'], (0,2) : ['(0,1)', '(1,2)'], (1,2) : ['(0,2)', '(1,3)'], (1,3) : ['(1,2)', '(2,3)']]
Optimal:  [(0,0) : ['(0,1)', '(1,0)'], (0,1) : ['(0,0)', '(0,2)'], (0,2) : ['(0,1)', '(1,2)'], (1,2) : ['(0,2)', '(1,3)'], (1,3) : ['(1,2)', '(2,3)']]


6

In [ ]:
def traverse_grid(node, goalList, frontier, explored): #function to traverse to other nodes.

# Does this work on a 2d grid when we can have a root top, bottom, left and right?
# keep track of coordinates
# build up frontier
# create a priority queue of nodes to visit

  #goal = grid("G")
  #heap = hq.heapify([node])

  explored.append(node)
  node.isExplored = True

  if node.value == 'g' or node.value == 'G':
    return node
  else:
    for adj in node.adjacent:
      if adj.pathcost > -1 and not adj.isExplored and pathcost(adj, node) < adj.pathcost:
        frontier.remove(adj)
        #Need to also update entire subtree's pathcosts
      if adj.pathcost > -1 and adj.isExplored and pathcost(adj, node) < adj.pathcost:
        adj.isExplored = False
        explored.remove(adj)

      if not adj.isExplored and not adj in frontier:
        adj.pathcost = pathcost(adj, node)
        adj.parent = node
        adj.totalcost = node_cost(adj, node, goalList)
        frontier.append(adj)

    frontier = sorted(frontier, key= lambda node : node.totalcost)
    return traverse_grid(frontier.pop(0), goalList, frontier, explored)

In [ ]:
def addToFrontier(node):
#adds a node to the frontier for later exploration
#only adds node if not already explored (check bool)
#Updates node if already on the frontier.


In [ ]:
def pathfinding(CSV, optimal, explored):
# call all functions and write to the files
  nodeArray = []
  startNode = None
  goalArray = []
  format_grid(CSV, nodeArray, startNode, goalArray)
  explored = []
  frontier = []
  goalNode = traverse_grid(startNode, goalArray, frontier, explored)

In [ ]:
def node_cost(node, parent, goalList):
#output cost to traverse to node
  
#The total cost of exploring the node
#Depends on the path taken to the node
#need to get lowest F value (node.left vs node.right)
# F = G + H
# g = the movement cost to move from S to a given square on the grid, following the path generated to get there. 
# h = the estimated movement cost to move from that given square on the grid to goal
#goal nodes stored in an array for easy access by h function

  step_cost = 0
  if node.value != 'g' and node.value != 'G' \
    and node.value != 's' and node.value != 'S':
    step_cost = int(node.value)

  path_cost = pathcost(node,parent)
  # print(path_cost)
  heuristic_cost = manhattan(node, goalList)
  return path_cost + heuristic_cost

currnode = node((1,2), 5)
parentnode = node((1,1), 3)
parentnode.pathcost = 12
goalList = [node((8,6), 'G'), node((5,3), 'G'), node((12,7), 'G')]
node_cost(currnode, parentnode, goalList)


22

In [ ]:
def pathcost(node, parent):
  step_cost = 0
  if node.value != 'g' and node.value != 'G' \
    and node.value != 's' and node.value != 'S':
    step_cost = int(node.value)

  return parent.pathcost + step_cost

currnode = node((1,2), 5)
parentnode = node((1,1), 3)
parentnode.pathcost = 12
pathcost(currnode, parentnode)

17

In [ ]:
def manhattan(node, goalList):
  goalHeuristics = []
  for goal in goalList:
    goalHeuristics.append(abs(node.coords[0] - goal.coords[0]) + abs(node.coords[1] - goal.coords[1]))
  return min(goalHeuristics)

manhattan(node((0,0),3), [node((3,5), 'G')])

24

In [ ]:
class node:
    def __init__(self, coords, value):
        self.coords = coords
        self.value = value
        self.pathcost = 0
        self.totalcost = 0
        self.adjacent = []
        self.parent = None
        self.isExplored = False
    def __repr__(self):
        return '(' + ','.join(map(str,self.coords)) + ') : ' + str(list(map(str, self.adjacent)))
    def __str__(self):
        return '(' + ','.join(map(str,self.coords)) + ')'
    def addAdjacent(self, node):
        if node.value != 'X' and node.value != 'x':
            self.adjacent.append(node)

In [ ]:

def format_grid(filePath, nodeArray, goalArray): #gets input text and formats it


  #Reads the input csv creates data structure for traversal. Outputs populated data structure.
  #Calculates the adjacent nodes for each spot in the grid and store in array
  #Output: array of arrays of nodes

  with open(filePath, encoding="utf8") as fp:
    row = 0
    line = fp.readline()
    while line:

      nodeArray.append([])
      processed = line.strip().split(',')
      for col in range(0, len(processed)):
        currnode = node((row, col), processed[col].strip())
        currnode.pathcost = -1
        nodeArray[row].append(currnode)
        if currnode.value == 's' or currnode.value == 'S':
          currnode.pathcost = 0
          startNode = currnode
        if currnode.value == 'g' or currnode.value == 'G':
          goalArray.append(currnode)

        if col > 0:
          leftnode = nodeArray[row][col-1]
          leftnode.addAdjacent(currnode)
          currnode.addAdjacent(leftnode)
        if row > 0:
          uppernode = nodeArray[row-1][col]
          uppernode.addAdjacent(currnode)
          currnode.addAdjacent(uppernode)

      row += 1
      line = fp.readline()

  # print(nodeArray[1][0].adjacent)
  return startNode

x = None
nodeArray = []
goalArray = []
start = format_grid('/input.txt', nodeArray, goalArray)
print(nodeArray)
print(start)
print(goalArray)

[[(0,0) : ['(0,1)', '(1,0)'], (0,1) : ['(0,0)', '(0,2)', '(1,1)'], (0,2) : ['(0,1)', '(0,3)', '(1,2)'], (0,3) : ['(0,2)', '(0,4)'], (0,4) : ['(0,3)', '(1,4)']], [(1,0) : ['(0,0)', '(1,1)', '(2,0)'], (1,1) : ['(1,0)', '(0,1)', '(1,2)', '(2,1)'], (1,2) : ['(1,1)', '(0,2)', '(2,2)'], (1,3) : ['(1,2)', '(0,3)', '(1,4)'], (1,4) : ['(0,4)', '(2,4)']], [(2,0) : ['(1,0)', '(2,1)', '(3,0)'], (2,1) : ['(2,0)', '(1,1)', '(2,2)'], (2,2) : ['(2,1)', '(1,2)'], (2,3) : ['(2,2)', '(2,4)'], (2,4) : ['(1,4)', '(3,4)']], [(3,0) : ['(2,0)', '(4,0)'], (3,1) : ['(3,0)', '(2,1)', '(4,1)'], (3,2) : ['(2,2)', '(4,2)'], (3,3) : ['(3,4)', '(4,3)'], (3,4) : ['(2,4)', '(4,4)']], [(4,0) : ['(3,0)', '(4,1)'], (4,1) : ['(4,0)', '(4,2)'], (4,2) : ['(4,1)', '(4,3)'], (4,3) : ['(4,2)', '(4,4)'], (4,4) : ['(4,3)', '(3,4)']]]
((2,2) : ['(2,1)', '(1,2)'], 3)
[(4,4) : ['(4,3)', '(3,4)']]
